Create images & videos out of saved generation data

# Imports

In [1]:
import sys
from pathlib import Path

import imageio
import torch
from IPython.display import HTML

sys.path.append(str(Path("..").resolve()))

from utils.misc import _normalize_elements_for_logging

# Experiment

In [2]:
root_experiments_path = Path("/", "projects", "static2dynamic", "Thomas", "experiments")

In [3]:
project_name = "GaussianProxy"
run_name = "eval_logging_tests"

In [4]:
project_path = root_experiments_path / project_name
assert project_path.exists(), f"Project path {project_path} does not exist."

run_path = project_path / run_name
assert run_path.exists(), f"Run path {run_path} does not exist."

# Show video of trajectories

In [5]:
proc0_vids_path = run_path / "saved_artifacts" / "proc_0" / "trajectories"
assert proc0_vids_path.exists(), f"Proc0 vids path {proc0_vids_path} does not exist."
all_trajs = list(proc0_vids_path.rglob("*.pt"))
print(f"Found {len(all_trajs)} trajectories.")
last_traj_path = all_trajs[-1]
print(f"Last trajectory: {last_traj_path.name}")
last_traj = torch.load(last_traj_path).cpu()
print(f"Loaded trajectory with shape {last_traj.shape}, dtype {last_traj.dtype} on cpu")

Found 3 trajectories.
Last trajectory: step_300.pt
Loaded trajectory with shape torch.Size([16, 20, 3, 128, 128]), dtype torch.float32 on cpu


In [6]:
nomrd_vids = _normalize_elements_for_logging(
    last_traj,
    ["image min-max", "video min-max", "[-1;1] raw", "[-1;1] clipped"],
)

In [22]:
image_index = 2

In [23]:
for norm_method, vid in nomrd_vids.items():
    print(
        f"Norm method: {norm_method} | saved data shape: {vid.shape} | selected video sample shape: {vid[image_index].shape}"
    )
    # Convert np array to video
    video_name = f"{norm_method}.mp4"
    imageio.mimwrite(video_name, vid[image_index].transpose(0, 2, 3, 1), fps=2)

    # Display the video with a specific width
    display(HTML(f'<video width="800" controls><source src="{video_name}" type="video/mp4"></video>'))

Norm method: image min-max | saved data shape: (16, 20, 3, 128, 128) | selected video sample shape: (20, 3, 128, 128)


Norm method: video min-max | saved data shape: (16, 20, 3, 128, 128) | selected video sample shape: (20, 3, 128, 128)


Norm method: [-1:1] raw | saved data shape: (16, 20, 3, 128, 128) | selected video sample shape: (20, 3, 128, 128)


Norm method: [-1:1] clipped | saved data shape: (16, 20, 3, 128, 128) | selected video sample shape: (20, 3, 128, 128)
